This notebook runs a pipeline with the following steps:
* Create a test Visibility dataset with two components and no beam.
* Write to disk in MSv2 format.
* Read MSv2 data into Visibility dataset.$^{*}$
* Predict model visibilities.$^{*}$
* Do bandpass calibration.$^{*}$
* Check for convergence.$^{*}$

\* xarray dataset map_blocks() is used to distribute frequency sub-bands across dask tasks.

The pipeline is called once with the default internal dask cluster,
then again with an externally generated cluster.

In [1]:
from time import perf_counter

from dask.distributed import LocalCluster
from ska_sdp_instrumental_calibration.processing_tasks.lsm import Component
from ska_sdp_instrumental_calibration.workflow.pipelines import (
    bandpass_calibration,
)

<frozen abc>:106: FutureWarning: xarray subclass Visibility should explicitly define __slots__


In [2]:
# Simple bandpass calibration with internally generated cluster
t0 = perf_counter()
bandpass_calibration.run(
    {
        "lsm": [
            Component(
                name="comp1", RAdeg=20.0, DEdeg=-30.0, flux=1.0, alpha=0.0
            ),
            Component(
                name="comp2", RAdeg=19.5, DEdeg=-29.9, flux=0.1, alpha=0.0
            ),
        ],
        "beam_type": "none",
        "ms_name": "demo.ms",
        "hdf5_name": "demo.hdf5",
    }
)
print(f"pipeline ran in {perf_counter() - t0:.1f} seconds")

2024-12-19 13:41:44,929 - workflow.pipeline_config - INFO - Simulating demo MSv2 input
2024-12-19 13:41:44,931 - workflow.pipeline_config - INFO - Predicting visibilities without a beam
2024-12-19 13:41:44,932 - workflow.pipeline_config - INFO - Generating a demo MSv2 Measurement Set demo.ms
2024-12-19 13:41:44,954 - workflow.utils - INFO - Using AA1-Low with 20 stations
2024-12-19 13:41:45,532 - processing_tasks.predict - INFO - No beam model used in predict
2024-12-19 13:41:45,726 - workflow.utils - INFO - Applying direction-independent gain corruptions
2024-12-19 13:41:46,300 - pipeline.bandpass_calibration - INFO - Starting pipeline with 16-channel chunks
2024-12-19 13:41:46,301 - pipeline.bandpass_calibration - INFO - No dask cluster supplied. Using LocalCluster
2024-12-19 13:41:47,140 - pipeline.bandpass_calibration - INFO - Will read from demo.ms in 16-channel chunks
2024-12-19 13:41:47,277 - pipeline.bandpass_calibration - INFO - Setting vis predict in 16-channel chunks
2024-12

pipeline ran in 4.7 seconds


In [3]:
# Simple bandpass calibration with externally generated cluster
t0 = perf_counter()
cluster = LocalCluster()
print(f"Sending scheduler address {cluster.scheduler_address}")
bandpass_calibration.run(
    {
        "dask_scheduler_address": cluster.scheduler_address,
        "lsm": [
            Component(
                name="comp1", RAdeg=20.0, DEdeg=-30.0, flux=1.0, alpha=0.0
            ),
            Component(
                name="comp2", RAdeg=19.5, DEdeg=-29.9, flux=0.1, alpha=0.0
            ),
        ],
        "beam_type": "none",
        "ms_name": "demo.ms",
        "hdf5_name": "demo.hdf5",
    }
)
print(f"pipeline ran in {perf_counter() - t0:.1f} seconds")

2024-12-19 13:41:50,354 - workflow.pipeline_config - INFO - Simulating demo MSv2 input
2024-12-19 13:41:50,355 - workflow.pipeline_config - INFO - Predicting visibilities without a beam
2024-12-19 13:41:50,356 - workflow.pipeline_config - INFO - Generating a demo MSv2 Measurement Set demo.ms
2024-12-19 13:41:50,381 - workflow.utils - INFO - Using AA1-Low with 20 stations
2024-12-19 13:41:50,413 - processing_tasks.predict - INFO - No beam model used in predict
2024-12-19 13:41:50,522 - workflow.utils - INFO - Applying direction-independent gain corruptions


Sending scheduler address tcp://127.0.0.1:39425


2024-12-19 13:41:51,157 - pipeline.bandpass_calibration - INFO - Starting pipeline with 16-channel chunks
2024-12-19 13:41:51,158 - pipeline.bandpass_calibration - INFO - Using existing dask cluster tcp://127.0.0.1:39425
2024-12-19 13:41:51,179 - pipeline.bandpass_calibration - INFO - Will read from demo.ms in 16-channel chunks
2024-12-19 13:41:51,251 - pipeline.bandpass_calibration - INFO - Setting vis predict in 16-channel chunks
2024-12-19 13:41:51,295 - pipeline.bandpass_calibration - INFO - Setting calibration in 16-channel chunks
2024-12-19 13:41:51,351 - pipeline.bandpass_calibration - INFO - Running graph and returning calibration solutions
<frozen abc>:106: FutureWarning: xarray subclass Visibility should explicitly define __slots__
<frozen abc>:106: FutureWarning: xarray subclass Visibility should explicitly define __slots__
<frozen abc>:106: FutureWarning: xarray subclass Visibility should explicitly define __slots__
<frozen abc>:106: FutureWarning: xarray subclass Visibilit

pipeline ran in 3.5 seconds
